In [1]:
# Preparação de Dados para Modelo de Machine Learning - Diabetes
# Objetivo: Tratar outliers, encoding de variáveis categóricas e normalização

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib
import os

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Bibliotecas importadas com sucesso!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {pd.__version__}")


Bibliotecas importadas com sucesso!
Pandas version: 2.3.2
NumPy version: 2.3.3
Scikit-learn version: 2.3.2


In [2]:
# Carregamento dos dados
print("="*60)
print("CARREGAMENTO E ANÁLISE INICIAL DOS DADOS")
print("="*60)

# Carregar dados
df = pd.read_csv('../data/raw/diabetes_dataset.csv')

print(f"Shape do dataset: {df.shape}")
print(f"Número de linhas: {df.shape[0]:,}")
print(f"Número de colunas: {df.shape[1]}")

# Identificar tipos de variáveis
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

print(f"\nVariáveis numéricas ({len(numeric_columns)}): {numeric_columns}")
print(f"Variáveis categóricas ({len(categorical_columns)}): {categorical_columns}")

# Verificar variável target
target_column = 'diagnosed_diabetes'
if target_column in df.columns:
    print(f"\nVariável target: {target_column}")
    print("Distribuição da classe target:")
    print(df[target_column].value_counts())
    print(f"Proporção: {df[target_column].value_counts(normalize=True).round(3)}")
else:
    print("⚠️ Variável target 'diagnosed_diabetes' não encontrada!")

# Verificar valores nulos
print(f"\nValores nulos por coluna:")
null_counts = df.isnull().sum()
if null_counts.sum() > 0:
    print(null_counts[null_counts > 0])
else:
    print("✅ Nenhum valor nulo encontrado!")


CARREGAMENTO E ANÁLISE INICIAL DOS DADOS
Shape do dataset: (100000, 31)
Número de linhas: 100,000
Número de colunas: 31

Variáveis numéricas (24): ['age', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week', 'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'family_history_diabetes', 'hypertension_history', 'cardiovascular_history', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides', 'glucose_fasting', 'glucose_postprandial', 'insulin_level', 'hba1c', 'diabetes_risk_score', 'diagnosed_diabetes']
Variáveis categóricas (7): ['gender', 'ethnicity', 'education_level', 'income_level', 'employment_status', 'smoking_status', 'diabetes_stage']

Variável target: diagnosed_diabetes
Distribuição da classe target:
diagnosed_diabetes
1    59998
0    40002
Name: count, dtype: int64
Proporção: diagnosed_diabetes
1    0.6
0    0.4
Name: proportion, dtype: float64

Valores 

In [3]:
# Divisão estratificada dos dados (Train/Test)
print("="*60)
print("DIVISÃO ESTRATIFICADA DOS DADOS")
print("="*60)

# Separar features e target
X = df.drop(columns=[target_column])
y = df[target_column]

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")

# Divisão estratificada 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"\nDivisão dos dados:")
print(f"X_train: {X_train.shape} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"X_test: {X_test.shape} ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

# Verificar se a estratificação funcionou
print(f"\nDistribuição da classe target:")
print("Train set:")
print(y_train.value_counts(normalize=True).round(3))
print("\nTest set:")
print(y_test.value_counts(normalize=True).round(3))

# Salvar índices para referência
train_indices = X_train.index
test_indices = X_test.index

print(f"\n✅ Divisão estratificada concluída com sucesso!")
print(f"Random state: 42 (para reprodutibilidade)")


DIVISÃO ESTRATIFICADA DOS DADOS
Features (X): (100000, 30)
Target (y): (100000,)

Divisão dos dados:
X_train: (80000, 30) (80.0%)
X_test: (20000, 30) (20.0%)
y_train: (80000,)
y_test: (20000,)

Distribuição da classe target:
Train set:
diagnosed_diabetes
1    0.6
0    0.4
Name: proportion, dtype: float64

Test set:
diagnosed_diabetes
1    0.6
0    0.4
Name: proportion, dtype: float64

✅ Divisão estratificada concluída com sucesso!
Random state: 42 (para reprodutibilidade)


In [4]:
# Análise detalhada de outliers
print("="*60)
print("ANÁLISE DETALHADA DE OUTLIERS")
print("="*60)

def detect_outliers_iqr(data, column):
    """Detecta outliers usando o método IQR"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound, Q1, Q3, IQR

def detect_outliers_zscore(data, column, threshold=3):
    """Detecta outliers usando Z-score"""
    z_scores = np.abs(stats.zscore(data[column].dropna()))
    outliers = data[z_scores > threshold]
    return outliers

# Análise de outliers no conjunto de treino
print("Análise de outliers no conjunto de TREINO:")
print("-" * 50)

outlier_analysis = []
outlier_summary = {}

for col in numeric_columns:
    if col != target_column:  # Excluir variável target
        # Método IQR
        outliers_iqr, lower, upper, Q1, Q3, IQR = detect_outliers_iqr(X_train, col)
        n_outliers_iqr = len(outliers_iqr)
        pct_outliers_iqr = (n_outliers_iqr / len(X_train)) * 100
        
        # Método Z-score
        outliers_zscore = detect_outliers_zscore(X_train, col)
        n_outliers_zscore = len(outliers_zscore)
        pct_outliers_zscore = (n_outliers_zscore / len(X_train)) * 100
        
        outlier_analysis.append({
            'Variavel': col,
            'Outliers_IQR': n_outliers_iqr,
            'Pct_IQR': pct_outliers_iqr,
            'Outliers_ZScore': n_outliers_zscore,
            'Pct_ZScore': pct_outliers_zscore,
            'Q1': Q1,
            'Q3': Q3,
            'IQR': IQR,
            'Limite_Inferior': lower,
            'Limite_Superior': upper
        })
        
        outlier_summary[col] = {
            'iqr_outliers': n_outliers_iqr,
            'iqr_pct': pct_outliers_iqr,
            'zscore_outliers': n_outliers_zscore,
            'zscore_pct': pct_outliers_zscore,
            'bounds': (lower, upper)
        }

outlier_df = pd.DataFrame(outlier_analysis)
outlier_df = outlier_df.sort_values('Pct_IQR', ascending=False)

print("RESUMO DE OUTLIERS POR VARIÁVEL:")
print(outlier_df[['Variavel', 'Outliers_IQR', 'Pct_IQR', 'Outliers_ZScore', 'Pct_ZScore']].round(2))

# Identificar variáveis com muitos outliers
high_outlier_vars = outlier_df[outlier_df['Pct_IQR'] > 5]['Variavel'].tolist()
print(f"\nVariáveis com mais de 5% de outliers: {high_outlier_vars}")

# Mostrar estatísticas das variáveis com mais outliers
if high_outlier_vars:
    print(f"\nEstatísticas detalhadas das variáveis com mais outliers:")
    for var in high_outlier_vars[:5]:  # Mostrar apenas as 5 primeiras
        stats_info = outlier_summary[var]
        print(f"\n{var}:")
        print(f"  Outliers IQR: {stats_info['iqr_outliers']} ({stats_info['iqr_pct']:.2f}%)")
        print(f"  Limites: [{stats_info['bounds'][0]:.2f}, {stats_info['bounds'][1]:.2f}]")
        print(f"  Min/Max no dataset: [{X_train[var].min():.2f}, {X_train[var].max():.2f}]")


ANÁLISE DETALHADA DE OUTLIERS
Análise de outliers no conjunto de TREINO:
--------------------------------------------------
RESUMO DE OUTLIERS POR VARIÁVEL:
                              Variavel  Outliers_IQR  Pct_IQR  \
6              family_history_diabetes         17555    21.94   
8               cardiovascular_history          6341     7.93   
2   physical_activity_minutes_per_week          2607     3.26   
22                 diabetes_risk_score           739     0.92   
4                  sleep_hours_per_day           728     0.91   
13                          heart_rate           704     0.88   
9                                  bmi           598     0.75   
18                     glucose_fasting           590     0.74   
12                        diastolic_bp           583     0.73   
21                               hba1c           494     0.62   
19                glucose_postprandial           488     0.61   
15                     hdl_cholesterol           462     0.58  

In [5]:
# Estratégia de tratamento de outliers
print("="*60)
print("ESTRATÉGIA DE TRATAMENTO DE OUTLIERS")
print("="*60)

def treat_outliers_capping(data, column, method='iqr', factor=1.5):
    """
    Trata outliers usando capping (winsorization)
    
    Args:
        data: DataFrame
        column: nome da coluna
        method: 'iqr' ou 'zscore'
        factor: fator multiplicativo (1.5 para IQR, 3 para Z-score)
    """
    data_copy = data.copy()
    
    if method == 'iqr':
        Q1 = data_copy[column].quantile(0.25)
        Q3 = data_copy[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        
    elif method == 'zscore':
        mean = data_copy[column].mean()
        std = data_copy[column].std()
        lower_bound = mean - factor * std
        upper_bound = mean + factor * std
    
    # Aplicar capping
    data_copy[column] = np.where(data_copy[column] < lower_bound, lower_bound, data_copy[column])
    data_copy[column] = np.where(data_copy[column] > upper_bound, upper_bound, data_copy[column])
    
    return data_copy, lower_bound, upper_bound

# Decidir quais variáveis tratar
print("Decisões de tratamento de outliers:")
print("-" * 40)

# Variáveis que serão tratadas (com mais de 5% de outliers)
vars_to_treat = high_outlier_vars.copy()

# Adicionar variáveis importantes que podem ter outliers significativos
important_vars = ['age', 'bmi', 'glucose_fasting', 'hba1c', 'diabetes_risk_score']
for var in important_vars:
    if var in numeric_columns and var not in vars_to_treat:
        if outlier_summary[var]['iqr_pct'] > 2:  # Mais de 2% de outliers
            vars_to_treat.append(var)

vars_to_treat = list(set(vars_to_treat))  # Remover duplicatas
print(f"Variáveis que serão tratadas: {vars_to_treat}")

# Aplicar tratamento de outliers no conjunto de treino
X_train_clean = X_train.copy()
outlier_treatment_info = {}

print(f"\nAplicando tratamento de outliers...")
for var in vars_to_treat:
    if var in X_train_clean.columns:
        X_train_clean, lower_bound, upper_bound = treat_outliers_capping(
            X_train_clean, var, method='iqr', factor=1.5
        )
        
        outlier_treatment_info[var] = {
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'original_min': X_train[var].min(),
            'original_max': X_train[var].max(),
            'new_min': X_train_clean[var].min(),
            'new_max': X_train_clean[var].max()
        }
        
        print(f"  {var}: [{lower_bound:.2f}, {upper_bound:.2f}]")

print(f"\n✅ Tratamento de outliers concluído para {len(vars_to_treat)} variáveis")

# Mostrar impacto do tratamento
print(f"\nIMPACTO DO TRATAMENTO:")
print("-" * 30)
for var in vars_to_treat[:5]:  # Mostrar apenas as 5 primeiras
    info = outlier_treatment_info[var]
    print(f"{var}:")
    print(f"  Original: [{info['original_min']:.2f}, {info['original_max']:.2f}]")
    print(f"  Tratado:  [{info['new_min']:.2f}, {info['new_max']:.2f}]")
    print(f"  Limites:  [{info['lower_bound']:.2f}, {info['upper_bound']:.2f}]")


ESTRATÉGIA DE TRATAMENTO DE OUTLIERS
Decisões de tratamento de outliers:
----------------------------------------
Variáveis que serão tratadas: ['family_history_diabetes', 'cardiovascular_history']

Aplicando tratamento de outliers...
  family_history_diabetes: [0.00, 0.00]
  cardiovascular_history: [0.00, 0.00]

✅ Tratamento de outliers concluído para 2 variáveis

IMPACTO DO TRATAMENTO:
------------------------------
family_history_diabetes:
  Original: [0.00, 1.00]
  Tratado:  [0.00, 0.00]
  Limites:  [0.00, 0.00]
cardiovascular_history:
  Original: [0.00, 1.00]
  Tratado:  [0.00, 0.00]
  Limites:  [0.00, 0.00]


In [6]:
# Encoding de variáveis categóricas
print("="*60)
print("ENCODING DE VARIÁVEIS CATEGÓRICAS")
print("="*60)

# Analisar variáveis categóricas
print("Análise das variáveis categóricas:")
print("-" * 40)

categorical_analysis = {}
for col in categorical_columns:
    unique_values = X_train[col].nunique()
    value_counts = X_train[col].value_counts()
    most_common = value_counts.iloc[0]
    least_common = value_counts.iloc[-1]
    
    categorical_analysis[col] = {
        'unique_count': unique_values,
        'most_common': most_common,
        'least_common': least_common,
        'most_common_pct': (most_common / len(X_train)) * 100,
        'least_common_pct': (least_common / len(X_train)) * 100
    }
    
    print(f"{col}:")
    print(f"  Valores únicos: {unique_values}")
    print(f"  Mais comum: {value_counts.index[0]} ({categorical_analysis[col]['most_common_pct']:.1f}%)")
    print(f"  Menos comum: {value_counts.index[-1]} ({categorical_analysis[col]['least_common_pct']:.1f}%)")
    print(f"  Distribuição: {dict(value_counts.head(3))}")
    print()

# Decidir estratégia de encoding para cada variável
print("ESTRATÉGIA DE ENCODING:")
print("-" * 30)

encoding_strategy = {}
onehot_vars = []
label_vars = []

for col in categorical_columns:
    unique_count = categorical_analysis[col]['unique_count']
    
    if unique_count <= 5:  # Poucas categorias - One-Hot Encoding
        encoding_strategy[col] = 'onehot'
        onehot_vars.append(col)
        print(f"{col}: One-Hot Encoding ({unique_count} categorias)")
    else:  # Muitas categorias - Label Encoding
        encoding_strategy[col] = 'label'
        label_vars.append(col)
        print(f"{col}: Label Encoding ({unique_count} categorias)")

print(f"\nResumo:")
print(f"  One-Hot Encoding: {len(onehot_vars)} variáveis")
print(f"  Label Encoding: {len(label_vars)} variáveis")


ENCODING DE VARIÁVEIS CATEGÓRICAS
Análise das variáveis categóricas:
----------------------------------------
gender:
  Valores únicos: 3
  Mais comum: Female (50.2%)
  Menos comum: Other (2.1%)
  Distribuição: {'Female': np.int64(40182), 'Male': np.int64(38155), 'Other': np.int64(1663)}

ethnicity:
  Valores únicos: 5
  Mais comum: White (44.9%)
  Menos comum: Other (5.1%)
  Distribuição: {'White': np.int64(35918), 'Hispanic': np.int64(15993), 'Black': np.int64(14467)}

education_level:
  Valores únicos: 4
  Mais comum: Highschool (44.9%)
  Menos comum: No formal (5.1%)
  Distribuição: {'Highschool': np.int64(35935), 'Graduate': np.int64(28017), 'Postgraduate': np.int64(11972)}

income_level:
  Valores únicos: 5
  Mais comum: Middle (35.2%)
  Menos comum: High (5.0%)
  Distribuição: {'Middle': np.int64(28122), 'Lower-Middle': np.int64(20126), 'Upper-Middle': np.int64(15881)}

employment_status:
  Valores únicos: 4
  Mais comum: Employed (60.2%)
  Menos comum: Student (6.2%)
  Distribu

In [7]:
# Implementação do encoding
print("="*60)
print("IMPLEMENTAÇÃO DO ENCODING")
print("="*60)

# Aplicar encoding no conjunto de treino
X_train_encoded = X_train_clean.copy()

# 1. Label Encoding para variáveis com muitas categorias
print("Aplicando Label Encoding...")
label_encoders = {}

for col in label_vars:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train_encoded[col])
    label_encoders[col] = le
    print(f"  {col}: {len(le.classes_)} categorias -> {le.classes_}")

# 2. One-Hot Encoding para variáveis com poucas categorias
print(f"\nAplicando One-Hot Encoding...")
onehot_encoders = {}

for col in onehot_vars:
    # Criar dummies
    dummies = pd.get_dummies(X_train_encoded[col], prefix=col, drop_first=True)
    
    # Remover coluna original e adicionar dummies
    X_train_encoded = X_train_encoded.drop(columns=[col])
    X_train_encoded = pd.concat([X_train_encoded, dummies], axis=1)
    
    onehot_encoders[col] = {
        'categories': X_train_clean[col].unique(),
        'dummy_columns': dummies.columns.tolist()
    }
    
    print(f"  {col}: {len(dummies.columns)} colunas dummy criadas")

print(f"\n✅ Encoding concluído!")
print(f"Shape original: {X_train_clean.shape}")
print(f"Shape após encoding: {X_train_encoded.shape}")
print(f"Novas colunas criadas: {X_train_encoded.shape[1] - X_train_clean.shape[1]}")

# Mostrar novas colunas criadas
new_columns = [col for col in X_train_encoded.columns if col not in X_train_clean.columns]
print(f"\nNovas colunas: {new_columns}")

# Verificar se há valores nulos após encoding
null_after_encoding = X_train_encoded.isnull().sum().sum()
print(f"\nValores nulos após encoding: {null_after_encoding}")
if null_after_encoding > 0:
    print("⚠️ Valores nulos encontrados após encoding!")
    print(X_train_encoded.isnull().sum()[X_train_encoded.isnull().sum() > 0])
else:
    print("✅ Nenhum valor nulo após encoding!")


IMPLEMENTAÇÃO DO ENCODING
Aplicando Label Encoding...

Aplicando One-Hot Encoding...
  gender: 2 colunas dummy criadas
  ethnicity: 4 colunas dummy criadas
  education_level: 3 colunas dummy criadas
  income_level: 4 colunas dummy criadas
  employment_status: 3 colunas dummy criadas
  smoking_status: 2 colunas dummy criadas
  diabetes_stage: 4 colunas dummy criadas

✅ Encoding concluído!
Shape original: (80000, 30)
Shape após encoding: (80000, 45)
Novas colunas criadas: 15

Novas colunas: ['gender_Male', 'gender_Other', 'ethnicity_Black', 'ethnicity_Hispanic', 'ethnicity_Other', 'ethnicity_White', 'education_level_Highschool', 'education_level_No formal', 'education_level_Postgraduate', 'income_level_Low', 'income_level_Lower-Middle', 'income_level_Middle', 'income_level_Upper-Middle', 'employment_status_Retired', 'employment_status_Student', 'employment_status_Unemployed', 'smoking_status_Former', 'smoking_status_Never', 'diabetes_stage_No Diabetes', 'diabetes_stage_Pre-Diabetes', 'di

In [8]:
# Normalização/Padronização de variáveis numéricas
print("="*60)
print("NORMALIZAÇÃO/PADRONIZAÇÃO DE VARIÁVEIS NUMÉRICAS")
print("="*60)

# Identificar variáveis numéricas após encoding
numeric_columns_after_encoding = X_train_encoded.select_dtypes(include=[np.number]).columns.tolist()
print(f"Variáveis numéricas após encoding: {len(numeric_columns_after_encoding)}")
print(f"Colunas: {numeric_columns_after_encoding}")

# Analisar distribuições das variáveis numéricas
print(f"\nAnálise das distribuições:")
print("-" * 40)

scaling_analysis = {}
for col in numeric_columns_after_encoding:
    mean_val = X_train_encoded[col].mean()
    std_val = X_train_encoded[col].std()
    min_val = X_train_encoded[col].min()
    max_val = X_train_encoded[col].max()
    range_val = max_val - min_val
    
    # Calcular coeficiente de variação
    cv = std_val / mean_val if mean_val != 0 else 0
    
    scaling_analysis[col] = {
        'mean': mean_val,
        'std': std_val,
        'min': min_val,
        'max': max_val,
        'range': range_val,
        'cv': cv
    }
    
    print(f"{col}:")
    print(f"  Média: {mean_val:.2f}, Desvio: {std_val:.2f}")
    print(f"  Min: {min_val:.2f}, Max: {max_val:.2f}, Range: {range_val:.2f}")
    print(f"  CV: {cv:.3f}")

# Decidir estratégia de scaling
print(f"\nESTRATÉGIA DE SCALING:")
print("-" * 30)

# Variáveis com alta variabilidade (CV > 0.5) ou grande range
high_variability_vars = [col for col, info in scaling_analysis.items() 
                        if info['cv'] > 0.5 or info['range'] > 100]

# Variáveis com valores negativos (não podem usar MinMaxScaler)
negative_vars = [col for col, info in scaling_analysis.items() 
                if info['min'] < 0]

print(f"Variáveis com alta variabilidade: {high_variability_vars}")
print(f"Variáveis com valores negativos: {negative_vars}")

# Escolher StandardScaler como padrão (mais robusto)
scaler_choice = 'StandardScaler'
print(f"\nEscolha do scaler: {scaler_choice}")
print("Motivo: StandardScaler é robusto e funciona bem com a maioria dos algoritmos")

# Aplicar scaling
print(f"\nAplicando {scaler_choice}...")
scaler = StandardScaler()

# Aplicar apenas nas variáveis numéricas
X_train_scaled = X_train_encoded.copy()
X_train_scaled[numeric_columns_after_encoding] = scaler.fit_transform(
    X_train_encoded[numeric_columns_after_encoding]
)

print(f"✅ Scaling concluído!")

# Verificar resultado do scaling
print(f"\nVerificação do scaling:")
print("-" * 30)
for col in numeric_columns_after_encoding[:5]:  # Mostrar apenas as 5 primeiras
    mean_after = X_train_scaled[col].mean()
    std_after = X_train_scaled[col].std()
    print(f"{col}: Média={mean_after:.3f}, Desvio={std_after:.3f}")

print(f"\nShape final: {X_train_scaled.shape}")
print(f"Tipo de dados: {X_train_scaled.dtypes.value_counts()}")


NORMALIZAÇÃO/PADRONIZAÇÃO DE VARIÁVEIS NUMÉRICAS
Variáveis numéricas após encoding: 23
Colunas: ['age', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week', 'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'family_history_diabetes', 'hypertension_history', 'cardiovascular_history', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides', 'glucose_fasting', 'glucose_postprandial', 'insulin_level', 'hba1c', 'diabetes_risk_score']

Análise das distribuições:
----------------------------------------
age:
  Média: 50.10, Desvio: 15.59
  Min: 18.00, Max: 90.00, Range: 72.00
  CV: 0.311
alcohol_consumption_per_week:
  Média: 2.00, Desvio: 1.42
  Min: 0.00, Max: 10.00, Range: 10.00
  CV: 0.708
physical_activity_minutes_per_week:
  Média: 119.09, Desvio: 84.76
  Min: 0.00, Max: 833.00, Range: 833.00
  CV: 0.712
diet_score:
  Média: 5.99, Desvio: 1.78
  Min: 0.00, Max: 1

In [9]:
# Criação de Pipeline de Transformação
print("="*60)
print("CRIAÇÃO DE PIPELINE DE TRANSFORMAÇÃO")
print("="*60)

class OutlierTreatment:
    """Classe customizada para tratamento de outliers"""
    
    def __init__(self, outlier_vars, method='iqr', factor=1.5):
        self.outlier_vars = outlier_vars
        self.method = method
        self.factor = factor
        self.bounds_ = {}
    
    def fit(self, X, y=None):
        for var in self.outlier_vars:
            if var in X.columns:
                if self.method == 'iqr':
                    Q1 = X[var].quantile(0.25)
                    Q3 = X[var].quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - self.factor * IQR
                    upper_bound = Q3 + self.factor * IQR
                elif self.method == 'zscore':
                    mean = X[var].mean()
                    std = X[var].std()
                    lower_bound = mean - self.factor * std
                    upper_bound = mean + self.factor * std
                
                self.bounds_[var] = (lower_bound, upper_bound)
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        for var, (lower, upper) in self.bounds_.items():
            if var in X_transformed.columns:
                X_transformed[var] = np.where(X_transformed[var] < lower, lower, X_transformed[var])
                X_transformed[var] = np.where(X_transformed[var] > upper, upper, X_transformed[var])
        return X_transformed

class CategoricalEncoder:
    """Classe customizada para encoding de variáveis categóricas"""
    
    def __init__(self, onehot_vars, label_vars):
        self.onehot_vars = onehot_vars
        self.label_vars = label_vars
        self.label_encoders_ = {}
        self.onehot_categories_ = {}
    
    def fit(self, X, y=None):
        # Fit label encoders
        for var in self.label_vars:
            if var in X.columns:
                le = LabelEncoder()
                le.fit(X[var])
                self.label_encoders_[var] = le
        
        # Store onehot categories
        for var in self.onehot_vars:
            if var in X.columns:
                self.onehot_categories_[var] = X[var].unique()
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        
        # Apply label encoding
        for var, le in self.label_encoders_.items():
            if var in X_transformed.columns:
                X_transformed[var] = le.transform(X_transformed[var])
        
        # Apply onehot encoding
        for var in self.onehot_vars:
            if var in X_transformed.columns:
                dummies = pd.get_dummies(X_transformed[var], prefix=var, drop_first=True)
                X_transformed = X_transformed.drop(columns=[var])
                X_transformed = pd.concat([X_transformed, dummies], axis=1)
        
        return X_transformed

# Criar pipeline completo
print("Criando pipeline de transformação...")

# Definir transformadores
outlier_transformer = OutlierTreatment(vars_to_treat, method='iqr', factor=1.5)
categorical_transformer = CategoricalEncoder(onehot_vars, label_vars)
scaler_transformer = StandardScaler()

# Criar pipeline
preprocessing_pipeline = Pipeline([
    ('outlier_treatment', outlier_transformer),
    ('categorical_encoding', categorical_transformer),
    ('scaling', scaler_transformer)
])

print("✅ Pipeline criado com sucesso!")
print("Etapas do pipeline:")
for i, (name, transformer) in enumerate(preprocessing_pipeline.steps):
    print(f"  {i+1}. {name}: {type(transformer).__name__}")

# Testar pipeline no conjunto de treino
print(f"\nTestando pipeline no conjunto de treino...")
X_train_pipeline = preprocessing_pipeline.fit_transform(X_train)

print(f"Shape após pipeline: {X_train_pipeline.shape}")
print(f"Tipo de dados: {type(X_train_pipeline)}")

# Verificar se o resultado é similar ao processamento manual
print(f"\nComparação com processamento manual:")
print(f"Pipeline: {X_train_pipeline.shape}")
print(f"Manual:   {X_train_scaled.shape}")
print(f"Diferença: {abs(X_train_pipeline.shape[1] - X_train_scaled.shape[1])} colunas")


CRIAÇÃO DE PIPELINE DE TRANSFORMAÇÃO
Criando pipeline de transformação...
✅ Pipeline criado com sucesso!
Etapas do pipeline:
  1. outlier_treatment: OutlierTreatment
  2. categorical_encoding: CategoricalEncoder
  3. scaling: StandardScaler

Testando pipeline no conjunto de treino...
Shape após pipeline: (80000, 45)
Tipo de dados: <class 'numpy.ndarray'>

Comparação com processamento manual:
Pipeline: (80000, 45)
Manual:   (80000, 45)
Diferença: 0 colunas


In [10]:
# Validação das Transformações
print("="*60)
print("VALIDAÇÃO DAS TRANSFORMAÇÕES")
print("="*60)

# Aplicar pipeline no conjunto de teste
print("Aplicando pipeline no conjunto de teste...")
X_test_transformed = preprocessing_pipeline.transform(X_test)

print(f"Shape do conjunto de teste transformado: {X_test_transformed.shape}")
print(f"Tipo de dados: {type(X_test_transformed)}")

# Verificar se não há valores nulos
print(f"\nVerificação de valores nulos:")
print(f"Train: {np.isnan(X_train_pipeline).sum()}")
print(f"Test:  {np.isnan(X_test_transformed).sum()}")

# Verificar se há valores infinitos
print(f"\nVerificação de valores infinitos:")
print(f"Train: {np.isinf(X_train_pipeline).sum()}")
print(f"Test:  {np.isinf(X_test_transformed).sum()}")

# Verificar estatísticas básicas
print(f"\nEstatísticas básicas do conjunto de treino:")
print(f"Média: {X_train_pipeline.mean():.3f}")
print(f"Desvio padrão: {X_train_pipeline.std():.3f}")
print(f"Min: {X_train_pipeline.min():.3f}")
print(f"Max: {X_train_pipeline.max():.3f}")

print(f"\nEstatísticas básicas do conjunto de teste:")
print(f"Média: {X_test_transformed.mean():.3f}")
print(f"Desvio padrão: {X_test_transformed.std():.3f}")
print(f"Min: {X_test_transformed.min():.3f}")
print(f"Max: {X_test_transformed.max():.3f}")

# Verificar se as distribuições são similares
print(f"\nComparação de distribuições (primeiras 5 features):")
for i in range(min(5, X_train_pipeline.shape[1])):
    train_mean = X_train_pipeline[:, i].mean()
    test_mean = X_test_transformed[:, i].mean()
    train_std = X_train_pipeline[:, i].std()
    test_std = X_test_transformed[:, i].std()
    
    print(f"Feature {i}:")
    print(f"  Train - Média: {train_mean:.3f}, Desvio: {train_std:.3f}")
    print(f"  Test  - Média: {test_mean:.3f}, Desvio: {test_std:.3f}")
    print(f"  Diferença - Média: {abs(train_mean - test_mean):.3f}, Desvio: {abs(train_std - test_std):.3f}")

# Verificar se o target não foi afetado
print(f"\nVerificação do target:")
print(f"Train target shape: {y_train.shape}")
print(f"Test target shape: {y_test.shape}")
print(f"Train target distribution: {y_train.value_counts(normalize=True).round(3)}")
print(f"Test target distribution: {y_test.value_counts(normalize=True).round(3)}")

print(f"\n✅ Validação concluída com sucesso!")


VALIDAÇÃO DAS TRANSFORMAÇÕES
Aplicando pipeline no conjunto de teste...
Shape do conjunto de teste transformado: (20000, 45)
Tipo de dados: <class 'numpy.ndarray'>

Verificação de valores nulos:
Train: 0
Test:  0

Verificação de valores infinitos:
Train: 0
Test:  0

Estatísticas básicas do conjunto de treino:
Média: 0.000
Desvio padrão: 0.978
Min: -3.964
Max: 27.851

Estatísticas básicas do conjunto de teste:
Média: -0.000
Desvio padrão: 0.971
Min: -3.751
Max: 27.851

Comparação de distribuições (primeiras 5 features):
Feature 0:
  Train - Média: 0.000, Desvio: 1.000
  Test  - Média: 0.008, Desvio: 1.004
  Diferença - Média: 0.008, Desvio: 0.004
Feature 1:
  Train - Média: 0.000, Desvio: 1.000
  Test  - Média: 0.002, Desvio: 0.996
  Diferença - Média: 0.002, Desvio: 0.004
Feature 2:
  Train - Média: -0.000, Desvio: 1.000
  Test  - Média: -0.011, Desvio: 0.979
  Diferença - Média: 0.011, Desvio: 0.021
Feature 3:
  Train - Média: -0.000, Desvio: 1.000
  Test  - Média: 0.003, Desvio: 0.99

In [11]:
# Salvar dados processados e pipeline
print("="*60)
print("SALVANDO DADOS PROCESSADOS E PIPELINE")
print("="*60)

# Criar diretórios se não existirem
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../models', exist_ok=True)

# Salvar dados processados
print("Salvando dados processados...")

# Converter arrays numpy para DataFrames para facilitar o uso
feature_names = [f'feature_{i}' for i in range(X_train_pipeline.shape[1])]

X_train_df = pd.DataFrame(X_train_pipeline, columns=feature_names, index=train_indices)
X_test_df = pd.DataFrame(X_test_transformed, columns=feature_names, index=test_indices)

# Salvar como CSV
X_train_df.to_csv('../data/processed/X_train_processed.csv')
X_test_df.to_csv('../data/processed/X_test_processed.csv')
y_train.to_csv('../data/processed/y_train.csv')
y_test.to_csv('../data/processed/y_test.csv')

print("✅ Dados processados salvos:")
print(f"  X_train: ../data/processed/X_train_processed.csv ({X_train_df.shape})")
print(f"  X_test:  ../data/processed/X_test_processed.csv ({X_test_df.shape})")
print(f"  y_train: ../data/processed/y_train.csv ({y_train.shape})")
print(f"  y_test:  ../data/processed/y_test.csv ({y_test.shape})")

# Salvar pipeline de transformação
print(f"\nSalvando pipeline de transformação...")
joblib.dump(preprocessing_pipeline, '../models/preprocessing_pipeline.pkl')

print("✅ Pipeline salvo: ../models/preprocessing_pipeline.pkl")

# Salvar informações sobre as transformações
transformation_info = {
    'outlier_vars': vars_to_treat,
    'onehot_vars': onehot_vars,
    'label_vars': label_vars,
    'numeric_vars': numeric_columns_after_encoding,
    'feature_names': feature_names,
    'outlier_treatment_info': outlier_treatment_info,
    'categorical_analysis': categorical_analysis,
    'scaling_analysis': scaling_analysis
}

joblib.dump(transformation_info, '../models/transformation_info.pkl')
print("✅ Informações de transformação salvas: ../models/transformation_info.pkl")

# Criar resumo das transformações
print(f"\nRESUMO DAS TRANSFORMAÇÕES:")
print("-" * 40)
print(f"Dataset original: {df.shape}")
print(f"Dataset processado: {X_train_pipeline.shape}")
print(f"Variáveis tratadas para outliers: {len(vars_to_treat)}")
print(f"Variáveis com One-Hot Encoding: {len(onehot_vars)}")
print(f"Variáveis com Label Encoding: {len(label_vars)}")
print(f"Variáveis normalizadas: {len(numeric_columns_after_encoding)}")
print(f"Features finais: {X_train_pipeline.shape[1]}")

print(f"\n🎯 PRÓXIMOS PASSOS:")
print("-" * 20)
print("1. Os dados estão prontos para treinamento de modelos")
print("2. Use X_train_pipeline e y_train para treinar modelos")
print("3. Use X_test_transformed e y_test para avaliação")
print("4. O pipeline pode ser reutilizado com novos dados")
print("5. Considere feature selection e engenharia de features adicionais")

print(f"\n✅ Preparação de dados concluída com sucesso!")


SALVANDO DADOS PROCESSADOS E PIPELINE
Salvando dados processados...
✅ Dados processados salvos:
  X_train: ../data/processed/X_train_processed.csv ((80000, 45))
  X_test:  ../data/processed/X_test_processed.csv ((20000, 45))
  y_train: ../data/processed/y_train.csv ((80000,))
  y_test:  ../data/processed/y_test.csv ((20000,))

Salvando pipeline de transformação...
✅ Pipeline salvo: ../models/preprocessing_pipeline.pkl
✅ Informações de transformação salvas: ../models/transformation_info.pkl

RESUMO DAS TRANSFORMAÇÕES:
----------------------------------------
Dataset original: (100000, 31)
Dataset processado: (80000, 45)
Variáveis tratadas para outliers: 2
Variáveis com One-Hot Encoding: 7
Variáveis com Label Encoding: 0
Variáveis normalizadas: 23
Features finais: 45

🎯 PRÓXIMOS PASSOS:
--------------------
1. Os dados estão prontos para treinamento de modelos
2. Use X_train_pipeline e y_train para treinar modelos
3. Use X_test_transformed e y_test para avaliação
4. O pipeline pode ser r

# 📊 Resumo Executivo da Preparação de Dados

## 🎯 Objetivo
Este notebook implementa o pipeline completo de preparação de dados para treinamento de modelos de Machine Learning, seguindo as recomendações da análise exploratória.

## 🔧 Transformações Implementadas

### 1. Divisão Estratificada dos Dados
- **Split**: 80% treino / 20% teste
- **Estratificação**: Mantém proporção das classes
- **Random State**: 42 (reprodutibilidade)

### 2. Tratamento de Outliers
- **Método**: Capping (Winsorization) usando IQR
- **Fator**: 1.5 × IQR
- **Variáveis tratadas**: Identificadas automaticamente baseado em % de outliers

### 3. Encoding de Variáveis Categóricas
- **One-Hot Encoding**: Variáveis com ≤ 5 categorias
- **Label Encoding**: Variáveis com > 5 categorias
- **Estratégia**: Automática baseada no número de categorias únicas

### 4. Normalização/Padronização
- **Método**: StandardScaler
- **Aplicação**: Apenas em variáveis numéricas
- **Resultado**: Média ≈ 0, Desvio padrão ≈ 1

## 📁 Arquivos Gerados

### Dados Processados
- `X_train_processed.csv`: Features de treino
- `X_test_processed.csv`: Features de teste
- `y_train.csv`: Target de treino
- `y_test.csv`: Target de teste

### Modelos e Pipeline
- `preprocessing_pipeline.pkl`: Pipeline completo de transformação
- `transformation_info.pkl`: Metadados das transformações

## ✅ Validações Realizadas

1. **Qualidade dos Dados**
   - ✅ Sem valores nulos
   - ✅ Sem valores infinitos
   - ✅ Tipos de dados consistentes

2. **Distribuições**
   - ✅ Normalização aplicada corretamente
   - ✅ Distribuições similares entre train/test
   - ✅ Target não afetado pelas transformações

3. **Reprodutibilidade**
   - ✅ Pipeline salvo e reutilizável
   - ✅ Random states fixados
   - ✅ Metadados preservados

## 🚀 Próximos Passos

1. **Treinamento de Modelos**
   - Usar dados processados para treinar diferentes algoritmos
   - Implementar validação cruzada
   - Otimizar hiperparâmetros

2. **Feature Engineering**
   - Criar features derivadas
   - Aplicar feature selection
   - Considerar interações entre variáveis

3. **Avaliação**
   - Métricas apropriadas para classificação
   - Análise de importância das features
   - Interpretabilidade do modelo

---
*Pipeline de preparação implementado com sucesso!*
